In [1]:
import pandas as pd
import numpy as np
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence
import gc

import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
from scipy.io import wavfile
file = "/home/peterr/macocu/task6_speech/data/00009874.flac.wav"

def get_duration(file:str) -> float:
    rate, data = wavfile.read(file)
    return len(data)/rate

get_duration(file)


pd.set_option("display.max_colwidth", 100)
f = "/home/nikolal/projects/parlaspeech/transcripts_normalized.txt"
df = pd.read_table(f, names=["unknown1", "path", "sentence"],
        #nrows=2000
        )

df["path"] = "/home/peterr/macocu/task6_speech/data/" + df.path.str.replace("flac/", "") + ".wav"
df["duration"] = df.path.apply(get_duration)
df = df.drop(columns="unknown1")
import os
df["filesize"] = df["path"].apply(os.path.getsize)
for char in ['*', ':', ';']:
    df["sentence"] = df.sentence.str.replace(char, "")
df.sort_values(by="filesize", ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df["to_edit"] = df.duration > 20
df.loc[df.to_edit, :].sample(5)
df["cuts"] = 0

2021-12-14 13:34:17,347 - INFO - NumExpr defaulting to 8 threads.


In [2]:
def splitter(file, MIN=5, MAX=10):
    def is_ok(pauses, duration, MIN=MIN, MAX=MAX):
        pauses = [0, *pauses, duration]
        durations = [MIN * 1000 <= e - s <= MAX * 1000 for s, e in zip(pauses[:-1], pauses[1:])]
        return all(durations)

    def _splitter(pauses, duration, MIN=MIN, MAX=MAX):
        from itertools import combinations, chain
        def powerset(iterable):
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
        all_combinations = list(powerset(pauses))
        logging.debug(f"Testing {len(all_combinations)} combinations....")
        for i, combination in enumerate(all_combinations):
            if i & 1024 == 0:
                gc.collect()
            if is_ok(combination, duration, MIN=MIN, MAX=MAX):
                return list(combination)
        logging.debug(f"No solution found so that {MIN=}s <= duration <= {MAX=}s.")
        return None

    def _splitter_heuristic(pauses, duration, MIN=MIN, MAX=MAX):
        pauses = [0, *pauses, duration]
        subsequent_pauses = [[s,e] for s, e in zip(pauses, pauses[1:]) if MIN <= e-s <= MAX]
        res = set()
        for s, e in subsequent_pauses:
            if s != 0:
                res.add(s)
            if e != duration:
                res.add(e)
        result = sorted(list(res))
        if _is_ok(pauses, duration, MIN=MIN, MAX=MAX):
            return result
        else:
            return None 
    audio = AudioSegment.from_wav(file)
    duration = audio.duration_seconds * 1000
    logging.debug(f"Duration: {duration/1000} s")
    res = None
    for silence in [1000, 800, 500, 300, 200]:
        logging.info(f"Testing silence {silence}")
        detected_silences = detect_silence(audio, min_silence_len=silence, silence_thresh=-40) # In seconds
        if detected_silences == []:
            logging.info(f"No silences detected")
            continue
        nr_of_silences = len(detected_silences)
        logging.debug(f"Got {nr_of_silences} silences.")
        centroids = np.array(detected_silences).mean(axis=1)
        centroids = centroids.tolist()
        res = None
        if nr_of_silences > 15:
            res = _splitter_heuristic(centroids, duration, MIN=MIN, MAX=MAX)
        else:
            res = _splitter(centroids, duration, MIN=MIN, MAX=MAX)
        if res != None:
            logging.debug(f" Success! Found splitting: {res}")
            break
    if res == None:
        raise Exception("No splitting was found.")
    centroids = np.array(res).tolist()
    cuts = [0, *centroids, duration]

    return cuts
    # new_filenames = list()
    # for i, (start, end) in enumerate(zip(cuts[0:],cuts[1:])):
    #     new_file = file.replace(".flac.wav", f"_{i:2}.flac.wav")
    #     segment = audio[start:end]
    #     logging.info(f"Exporting segment nr. {i+1} as f{new_file}")
    #     segment.export(new_file, format="wav")
    #     new_filenames.append(new_file)
    # return new_filenames



In [8]:
df = pd.read_csv("/home/peterr/macocu/task6_speech/8_results.csv")
#df["cuts"] = np.nan
df[df.to_edit]["cuts"].unique()
0 in df.cuts

True

In [9]:

i = 0
for row in df.loc[df.to_edit,].iterrows():
    if row[1]["cuts"] == 0:
        logging.debug(f"File {row[1]['path']} was already processed.")
        continue
    try:
        file = row[1]["path"]
        cuts = splitter(file)
        df.loc[row[0], "cuts"] = str(cuts)
        logging.info(f"Finished number {i}.")
        i += 1
    except:
        logging.error(f"Could not calculate splits for {row[1]['path']}...")
        continue
    finally:
        df.to_csv("/home/peterr/macocu/task6_speech/8_results.csv", index = False)

2021-12-14 13:35:44,314 - INFO - Testing silence 1000
2021-12-14 13:35:44,953 - INFO - Testing silence 800
2021-12-14 13:35:45,655 - INFO - Testing silence 500
2021-12-14 13:35:51,440 - INFO - Testing silence 300
2021-12-14 13:35:52,872 - INFO - Finished number 0.
2021-12-14 13:35:53,012 - INFO - Testing silence 1000
2021-12-14 13:35:53,562 - INFO - No silences detected
2021-12-14 13:35:53,563 - INFO - Testing silence 800
2021-12-14 13:35:54,085 - INFO - Testing silence 500
2021-12-14 13:35:55,153 - INFO - Finished number 1.
2021-12-14 13:35:55,292 - INFO - Testing silence 1000
2021-12-14 13:35:55,843 - INFO - No silences detected
2021-12-14 13:35:55,843 - INFO - Testing silence 800
2021-12-14 13:35:56,313 - INFO - No silences detected
2021-12-14 13:35:56,315 - INFO - Testing silence 500
2021-12-14 13:35:56,662 - INFO - No silences detected
2021-12-14 13:35:56,663 - INFO - Testing silence 300
2021-12-14 13:35:57,171 - INFO - Finished number 2.
2021-12-14 13:35:57,311 - INFO - Testing s

In [3]:
file

'/home/peterr/macocu/task6_speech/data/00009874.flac.wav'